In [ ]:
import json
import os
import sys

import dotenv
import googlemaps
import pandas as pd
from openai import OpenAI

dotenv.load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_PLACES_API_KEY")
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")

CURRENT_DIR = %pwd
ROOT_DIR = os.path.dirname(CURRENT_DIR)
SRC_DIR = os.path.join(ROOT_DIR, "src")

sys.path.append(SRC_DIR)

In [ ]:
from searches.google_places import GoogleMapsAPI, GooglePlacesAPI
from searches.util import (
    extract_city,
    get_city_center_coordinates,
    miles_to_meters,
    meters_to_miles,
)
from text_util import clean_text, parse_itenerary_content

In [ ]:
inputs = {
    "location": "South Beach Miami, FL",
    "number_of_people": 6,
    "date": "November 2026",
    "duration_days": 3,
    "group_type": "bachelorette party",
    "description": (
        "include boutique hotel options must 4 stars higher onsite spa beach clubs djs"
        "day high end nightclubs list least six restaurant options dinner include least "
        "one nice steakhouse one nice sushi restaurant"
    ),
}

In [ ]:
PROMPT_START = """Create itinerary: provide only names
for places listed in google places for breakfast,
morning activity, lunch, afternoon activity, dinner and
evening activity. return only the day and the name:
""".replace(
    "\n", " "
)

PROMPT_TEMPLATE = f"""“{inputs['duration_days']}” day
“{inputs['group_type']}” to “{inputs['location']}” for “{inputs['number_of_people']}”
people in “{inputs['date']}” that enjoy “{inputs['description']}”
"""

prompt = f"{PROMPT_START}{clean_text(PROMPT_TEMPLATE)}"
print(prompt)

**Optimize Prompt Length**

The cleaned_text contains a lot of detailed requirements for the itinerary. While detail is good, ensure that every word is necessary to fulfill the request. Extra tokens increase cost without adding value.

**Adjust Temperature**

A temperature of 1 is the most creative setting, leading to more varied outputs. However, for generating an itinerary based on specific criteria, a lower temperature might produce more consistent and focused results, potentially reducing the need for additional tokens to clarify or correct the output.

**Refine Max Tokens**

The max_tokens setting determines the maximum length of the generated response. If you find that the responses are consistently shorter than your limit, you can lower max_tokens to save on costs. Analyze the average length of the responses you're getting and adjust accordingly.

**Use top_p Wisely**

Setting top_p=1 means the model considers all possible next tokens at each step, which is fine for maximizing creativity but might not be necessary for your use case. Adjusting top_p to a lower value could make the model's responses more focused and concise, potentially reducing token usage.

**Penalties**

You've set frequency_penalty and presence_penalty to 0, which is neutral and doesn't influence the model's token choices. Depending on the variety you need in the responses, tweaking these values can help manage repetition and ensure the uniqueness of the places listed, potentially making the responses more efficient.


In [ ]:
client = OpenAI(api_key=OPEN_AI_API_KEY)


response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)
print(response.choices[0].message.content.split("\n"))

# Extract token usage from the response
completion_tokens = response.usage.completion_tokens
prompt_tokens = response.usage.prompt_tokens
total_tokens = completion_tokens + prompt_tokens

# Cost estimation for 1,000,000 requests
total_requests = 1_000_000
total_tokens_for_all_requests = total_tokens * total_requests

training_cost_per_million_tokens = 8.00
input_usage_cost_per_million_tokens = 3.00
output_usage_cost_per_million_tokens = 6.00

total_cost = (total_tokens_for_all_requests / 1_000_000) * (
    training_cost_per_million_tokens
    + input_usage_cost_per_million_tokens
    + output_usage_cost_per_million_tokens
)

print(f"Estimated total cost for {total_requests} requests: ${total_cost:.2f}")

In [ ]:
import typing as T
import re
def parse_itenerary_day(lines: T.List[str]) -> T.List[T.Tuple[str, str]]:
    day_plan = []
    # This pattern is designed to capture two groups separated by various delimiters
    pattern = re.compile(r"\-?\s*([\w\s]+?)\s*(?::|at|-)\s*([\w\s'&]+)")

    for line in lines:
        match = pattern.match(line)
        if match:
            activity_type, place = match.groups()
            activity_type = activity_type.strip()
            place = place.split("(")[0].strip()  # Removes anything within parentheses
            day_plan.append((activity_type, place))
        else:
            print(f"Could not parse line: {line}")

    return day_plan


def parse_itenerary_content(content: str) -> T.List[T.Dict[str, str]]:
    # Split the content into days and activities
    days = content.split("\n\n")
    data = []
    for day in days:
        lines = day.split("\n")
        day_number = lines[0].split(" ")[1]
        day_plan = parse_itenerary_day(lines[1:])
        for activity_type, place in day_plan:
            data.append(
                {"Day": day_number, "Activity Type": activity_type, "Place": place}
            )

    return data

In [ ]:
# Extract the itinerary content from the response
itinerary_content = response.choices[0].message.content
data = parse_itenerary_content(itinerary_content)
df = pd.DataFrame(data)

print(df)

In [ ]:
google_places = GooglePlacesAPI(api_key=GOOGLE_API_KEY)
google_maps = GoogleMapsAPI(api_key=GOOGLE_API_KEY)
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

In [ ]:
city_coordinates = get_city_center_coordinates(inputs["location"])
print(f"{inputs['location']} coordinates: {city_coordinates}")
keyword = "breakfast"

itinerary_place_details = []
nearby_place_details = {}
for index, row in df.iterrows():
    try:
        result = gmaps.places(query=row["Place"], location=city_coordinates)
    except:
        print(f"Unable to get places info for {row['Place']}")
        continue

    if not result or result.get("status") != "OK":
        print(f"Unable to get places info for {row['Place']}")
        continue

    place_result = result["results"][0]
    itinerary_place_details.append(place_result)

    radius_meters = miles_to_meters(1.0)

    print(f"Getting nearby places for {row['Place']} at {place_result['geometry']['location']}")
    try:
        nearby_places = gmaps.places_nearby(
            location=place_result["geometry"]["location"],
            radius=radius_meters,
            keyword=keyword if keyword else None,
            type=place_result["types"][0] if place_result["types"] else None,
        )
    except:
        print(f"Unable to get nearby places info for {row['Place']}")
        continue

    if not nearby_places or nearby_places.get("status") != "OK":
        print(f"Unable to get nearby places info for {row['Place']}")
        continue

    nearby_place_details[row["Place"]] = nearby_places["results"]


In [ ]:
print(f"Num results: {len(itinerary_place_details)}")
for item in itinerary_place_details:
    print(item["name"])
    nearby_names = [i["name"] for i in nearby_place_details.get(item["name"], [])]
    for name in nearby_names:
        print(f"  - {name}")